# EU HICP Package Holidays Price Forecast - Phase 4: Model Development & July 2025 Forecasting

## Overview
This notebook implements **Phase 4** of the EU HICP Package Holidays Price Forecast project, focusing on comprehensive model development and ensemble forecasting for **July 2025**.

### Objectives
1. **Seasonal Adjustment**: Apply STL decomposition to create seasonally adjusted series
2. **Feature Engineering**: Build comprehensive feature store with 80+ features
3. **Model Development**: Implement ensemble approach with ARIMA/SARIMA, ML models
4. **Validation Framework**: Out-of-sample testing and cross-validation
5. **July 2025 Forecast**: Generate final ensemble forecast with confidence intervals
6. **Interactive Visualization**: Plotly-based dashboards and analysis

### Target
**Forecast the seasonally adjusted month-on-month percentage change (MoM% SA) in the HICP index for package holidays in July 2025.**

In [ ]:
# Import required libraries
import polars as pl
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
import json

# Time series and ML models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

# Project modules
import sys
sys.path.append('.')
from seasonal_adjustment import SeasonalAdjuster
from feature_engineering import FeatureEngineer

# Configure settings
pl.Config.set_tbl_rows(20)
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

## Step 1: Load and Prepare Data

In [ ]:
# Load or create sample data
try:
    df = pl.read_parquet('data/clean_long_format.parquet')
    print(f"✓ Loaded existing data: {len(df)} observations")
except:
    print("Creating sample data for demonstration...")
    
    # Generate sample HICP data with seasonal patterns
    dates = pl.date_range(datetime(2010, 1, 1), datetime(2024, 12, 31), interval='1mo')
    n_obs = len(dates)
    
    np.random.seed(42)
    trend = np.linspace(100, 120, n_obs)
    seasonal = 5 * np.sin(2 * np.pi * np.arange(n_obs) / 12)
    noise = np.random.normal(0, 2, n_obs)
    
    eu_values = trend + seasonal + noise
    germany_values = trend * 0.95 + seasonal * 0.8 + np.random.normal(0, 1.5, n_obs)
    
    eu_df = pl.DataFrame({
        'date': dates, 'value': eu_values,
        'series_name': ['eu_package_holidays'] * n_obs,
        'series_id': ['CP96EAMM'] * n_obs
    })
    
    germany_df = pl.DataFrame({
        'date': dates, 'value': germany_values,
        'series_name': ['germany_package_holidays'] * n_obs,
        'series_id': ['CP96DEMM'] * n_obs
    })
    
    df = pl.concat([eu_df, germany_df])
    print(f"✓ Sample data created: {len(df)} observations")

print(f"Series: {df['series_name'].unique().to_list()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

## Step 2: Seasonal Adjustment & Feature Engineering

In [ ]:
# Apply seasonal adjustment and feature engineering
print("Applying seasonal adjustment...")

# Initialize components
adjuster = SeasonalAdjuster(method='stl')
engineer = FeatureEngineer()

# Process each series
adjusted_series = []
for series_name in df['series_name'].unique():
    try:
        adjusted_df = adjuster.adjust_series(df, series_name)
        adjusted_series.append(adjusted_df)
        print(f"✓ Processed {series_name}")
    except Exception as e:
        print(f"❌ Failed {series_name}: {e}")

# Combine adjusted series
if adjusted_series:
    seasonally_adjusted_df = pl.concat(adjusted_series)
    print(f"✓ Seasonal adjustment complete: {len(seasonally_adjusted_df)} observations")
else:
    seasonally_adjusted_df = df
    print("❌ Using original data without seasonal adjustment")

# Build feature store
print("\nBuilding feature store...")
feature_store = engineer.build_feature_store(seasonally_adjusted_df)

# Calculate target variable (MoM% SA)
feature_store = feature_store.sort(['series_name', 'date']).with_columns([
    (pl.col('value_sa' if 'value_sa' in feature_store.columns else 'value')
     .pct_change().over('series_name') * 100).alias('mom_pct_sa')
])

print(f"✓ Feature store complete: {feature_store.shape}")
print(f"  Features: {feature_store.width}")
print(f"  Target variable: mom_pct_sa")

## Step 3: Interactive Visualization

In [ ]:
# Visualize seasonal decomposition
def plot_seasonal_decomposition(df, series_name):
    series_data = df.filter(pl.col('series_name') == series_name).sort('date')
    if series_data.is_empty():
        return None
    
    plot_data = series_data.to_pandas()
    
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=[f'{series_name} - Original vs SA', 'Seasonal', 'Target (MoM% SA)'],
        vertical_spacing=0.1
    )
    
    # Original vs seasonally adjusted
    fig.add_trace(go.Scatter(
        x=plot_data['date'], y=plot_data['value'],
        name='Original', line=dict(color='blue')
    ), row=1, col=1)
    
    if 'value_sa' in plot_data.columns:
        fig.add_trace(go.Scatter(
            x=plot_data['date'], y=plot_data['value_sa'],
            name='Seasonally Adjusted', line=dict(color='red', dash='dash')
        ), row=1, col=1)
    
    # Seasonal component
    if 'seasonal' in plot_data.columns:
        fig.add_trace(go.Scatter(
            x=plot_data['date'], y=plot_data['seasonal'],
            name='Seasonal', line=dict(color='orange'), showlegend=False
        ), row=2, col=1)
    
    # Target variable
    fig.add_trace(go.Scatter(
        x=plot_data['date'], y=plot_data['mom_pct_sa'],
        name='MoM% SA', line=dict(color='green'), showlegend=False
    ), row=3, col=1)
    
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3, col=1)
    
    fig.update_layout(height=800, title=f'Analysis - {series_name}', hovermode='x unified')
    return fig

# Plot for each series
for series_name in feature_store['series_name'].unique():
    fig = plot_seasonal_decomposition(feature_store, series_name)
    if fig:
        fig.show()

## Step 4: Model Development & July 2025 Forecasting

In [ ]:
# Modeling class for interactive use
class JupyterModeler:
    def __init__(self, target_col='mom_pct_sa'):
        self.target_col = target_col
        self.results = {}
    
    def prepare_data(self, df, series_name):
        series_data = df.filter(pl.col('series_name') == series_name).sort('date')
        pandas_data = series_data.to_pandas().set_index('date')
        
        target = pandas_data[self.target_col].dropna()
        
        exclude_cols = ['series_name', 'series_id', self.target_col]
        feature_cols = [col for col in pandas_data.columns if col not in exclude_cols]
        features = pandas_data[feature_cols].loc[target.index]
        
        return target, features
    
    def fit_sarima(self, target):
        try:
            model = SARIMAX(target, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
            fitted = model.fit(disp=False)
            forecast = fitted.forecast(steps=1)[0]
            return {'forecast': forecast, 'aic': fitted.aic, 'success': True}
        except Exception as e:
            return {'success': False, 'error': str(e)}
    
    def fit_ml_models(self, target, features):
        common_idx = target.index.intersection(features.index)
        y = target.loc[common_idx]
        X = features.loc[common_idx].select_dtypes(include=[np.number])
        X = X.loc[:, X.isnull().mean() < 0.5].fillna(X.median())
        
        if len(y) < 20 or X.shape[1] == 0:
            return {}
        
        models = {
            'RandomForest': RandomForestRegressor(n_estimators=50, random_state=42),
            'XGBoost': xgb.XGBRegressor(n_estimators=50, random_state=42),
            'Ridge': Ridge(alpha=1.0)
        }
        
        results = {}
        for name, model in models.items():
            try:
                if name == 'Ridge':
                    scaler = RobustScaler()
                    X_scaled = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
                    model.fit(X_scaled, y)
                    forecast = model.predict(X_scaled.iloc[-1:].values)[0]
                else:
                    model.fit(X, y)
                    forecast = model.predict(X.iloc[-1:].values)[0]
                
                mse = mean_squared_error(y, model.predict(X_scaled if name == 'Ridge' else X))
                results[name] = {'forecast': forecast, 'mse': mse, 'success': True}
            except Exception as e:
                results[name] = {'success': False, 'error': str(e)}
        
        return results
    
    def create_ensemble_forecast(self, individual_forecasts, performance_metrics):
        if not individual_forecasts:
            return None
        
        # Weighted average based on inverse MSE
        weights = {}
        total_weight = 0
        
        for model in individual_forecasts:
            if model in performance_metrics:
                weight = 1 / (performance_metrics[model] + 1e-8)
            else:
                weight = 1
            weights[model] = weight
            total_weight += weight
        
        # Normalize weights
        weights = {model: w/total_weight for model, w in weights.items()}
        
        # Calculate ensemble forecast
        ensemble_forecast = sum(individual_forecasts[model] * weights[model] for model in individual_forecasts)
        
        return {
            'ensemble_forecast': ensemble_forecast,
            'individual_forecasts': individual_forecasts,
            'weights': weights
        }
    
    def forecast_july_2025(self, df, series_name):
        print(f"\nForecasting July 2025 for {series_name}...")
        
        target, features = self.prepare_data(df, series_name)
        
        individual_forecasts = {}
        performance_metrics = {}
        
        # SARIMA
        sarima_result = self.fit_sarima(target)
        if sarima_result['success']:
            individual_forecasts['SARIMA'] = sarima_result['forecast']
            performance_metrics['SARIMA'] = sarima_result['aic']
            print(f"  SARIMA forecast: {sarima_result['forecast']:.3f}%")
        
        # ML Models
        ml_results = self.fit_ml_models(target, features)
        for model_name, result in ml_results.items():
            if result['success']:
                individual_forecasts[model_name] = result['forecast']
                performance_metrics[model_name] = result['mse']
                print(f"  {model_name} forecast: {result['forecast']:.3f}%")
        
        # Ensemble
        ensemble_result = self.create_ensemble_forecast(individual_forecasts, performance_metrics)
        
        if ensemble_result:
            print(f"  \n🎯 ENSEMBLE FORECAST: {ensemble_result['ensemble_forecast']:.3f}%")
            print(f"  Model weights: {ensemble_result['weights']}")
            
            # Calculate confidence interval
            forecast_values = list(individual_forecasts.values())
            forecast_std = np.std(forecast_values)
            
            ensemble_result['confidence_intervals'] = {
                'lower_95': ensemble_result['ensemble_forecast'] - 1.96 * forecast_std,
                'upper_95': ensemble_result['ensemble_forecast'] + 1.96 * forecast_std
            }
            
            print(f"  95% CI: [{ensemble_result['confidence_intervals']['lower_95']:.3f}%, {ensemble_result['confidence_intervals']['upper_95']:.3f}%]")
        
        return ensemble_result

# Initialize modeler
modeler = JupyterModeler()

# Generate forecasts for all series
forecast_results = {}
for series_name in feature_store['series_name'].unique():
    result = modeler.forecast_july_2025(feature_store, series_name)
    if result:
        forecast_results[series_name] = result

## Step 5: Final Results & Visualization

In [ ]:
# Create comprehensive forecast visualization
def create_forecast_dashboard(df, forecast_results):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Historical MoM% SA Changes',
            'July 2025 Forecasts',
            'Model Weights',
            'Forecast Confidence Intervals'
        ],
        specs=[[{"secondary_y": False}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )
    
    # Historical data
    for series_name in df['series_name'].unique():
        series_data = df.filter(pl.col('series_name') == series_name).sort('date')
        plot_data = series_data.to_pandas()
        
        fig.add_trace(go.Scatter(
            x=plot_data['date'], y=plot_data['mom_pct_sa'],
            name=series_name, mode='lines+markers'
        ), row=1, col=1)
    
    # July 2025 forecasts
    series_names = list(forecast_results.keys())
    forecasts = [forecast_results[s]['ensemble_forecast'] for s in series_names]
    
    fig.add_trace(go.Bar(
        x=series_names, y=forecasts,
        name='July 2025 Forecast',
        marker_color='red'
    ), row=1, col=2)
    
    # Model weights (for first series)
    if series_names:
        first_series = series_names[0]
        weights = forecast_results[first_series]['weights']
        
        fig.add_trace(go.Bar(
            x=list(weights.keys()), y=list(weights.values()),
            name='Model Weights',
            marker_color='blue'
        ), row=2, col=1)
    
    # Confidence intervals
    lower_ci = [forecast_results[s]['confidence_intervals']['lower_95'] for s in series_names]
    upper_ci = [forecast_results[s]['confidence_intervals']['upper_95'] for s in series_names]
    
    fig.add_trace(go.Bar(
        x=series_names, y=lower_ci,
        name='Lower 95% CI',
        marker_color='lightblue'
    ), row=2, col=2)
    
    fig.add_trace(go.Bar(
        x=series_names, y=upper_ci,
        name='Upper 95% CI',
        marker_color='lightcoral'
    ), row=2, col=2)
    
    fig.update_layout(
        height=800,
        title='EU HICP Package Holidays - July 2025 Forecast Dashboard',
        showlegend=True
    )
    
    return fig

# Create and show dashboard
if forecast_results:
    dashboard = create_forecast_dashboard(feature_store, forecast_results)
    dashboard.show()
    
    # Print final summary
    print("\n" + "="*80)
    print("JULY 2025 FORECAST SUMMARY")
    print("="*80)
    
    for series_name, result in forecast_results.items():
        print(f"\n{series_name.upper()}:")
        print(f"  Ensemble Forecast: {result['ensemble_forecast']:.3f}%")
        print(f"  95% Confidence Interval: [{result['confidence_intervals']['lower_95']:.3f}%, {result['confidence_intervals']['upper_95']:.3f}%]")
        print(f"  Individual Forecasts: {result['individual_forecasts']}")
        print(f"  Model Weights: {result['weights']}")
    
    print("\n🎯 FORECASTING COMPLETE!")
    print("The ensemble model has successfully generated July 2025 forecasts for EU HICP Package Holidays.")
else:
    print("❌ No forecast results generated. Please check the data and model fitting.")